In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020', 'USZ 2018', 'USZ 2019', 'USZ 2019_30_04_2020', 'KSSG 2021', 'KSSG Linth 2019', 'KSSG Toggenburg 2019', 'KSSG SRRWS 2019'])

In [3]:
file = FILES_TO_ANALYZE['KSSG SRRWS 2019']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Linth_Toggenburg_SRRWS_2019.xlsx', hospital_name_db='KSSG SRRWS', year='2019', sheets=['Änderungen SRRWS_2019'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 48 cases for KSSG SRRWS 2019
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [6]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,2010216467,971401A627916522,W,67,16,2,I27D,M7285,M200,"D62,L0311",D648,<NA>,<NA>
1,2010230894,0FD7D457C22B4ECD,M,75,10,3,X06B,S510,M100,R18,<NA>,<NA>,<NA>
2,2010233003,D6D96CE0A68739FB,W,70,14,3,G21B,K432,M200,D62,D649,<NA>,<NA>
3,2010234179,FB8869E03D85B3F5,M,59,7,3,H62B,K868,M100,I5014,I509,<NA>,<NA>
4,2070263179,A8239CB0553D4656,W,79,8,3,F12D,I441,M100,J9580,<NA>,<NA>,<NA>


# match to the database


In [7]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['2010216467', '2010230894', '2010233003', '2010234179',
       '2070263179', '2010220228', '2010228707', '2010229560',
       '2010230678', '2070282416', '2070287841', '2070284708',
       '2010234918', '2010234948', '2070285115', '2010227097',
       '2010230191', '2010231388', '2070264552', '2010222584',
       '2010224862', '2010231874', '2010236987', '2010238492',
       '2010242300', '2070268669', '2010220656', '2010227288',
       '2070265027', '2010222118', '2010225191', '2010229750',
       '2010231195', '2010231435', '2070273678', '2010234290',
       '2010237009', '2070264374', '2070267501', '2070290212',
       '2070280035', '2010217844', '2010221707', '2010222228',
       '2010231467', '2010236020', '2070280704', '2070288570'],
      dtype=object)

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,0.706,294901,10,2010216467,971401A627916522,67,0,W,16,4,0,0,0,2019-01-18,01,2019-02-03,00,I27D,I27,2
1,1.302,293076,10,2010217844,4E884A43EE05202E,50,0,W,7,5,0,0,0,2019-02-28,01,2019-03-07,00,N04A,N04,0
2,1.793,293971,10,2010220228,0684D4B0A48A69C8,60,0,M,14,4,0,0,0,2019-04-16,01,2019-04-30,00,G18B,G18,3
3,0.502,300112,10,2010220656,3EB7CE71A578B8BE,83,0,W,4,3,0,0,0,2019-03-07,01,2019-03-11,00,F72B,F72,2
4,0.888,296788,10,2010221707,29E59C19E0C812A5,85,0,M,3,3,0,0,0,2019-03-22,01,2019-03-25,00,F62C,F62,3


In [9]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 48 out of 48 revised cases from DtoD that are matched with the database for KSSG SRRWS 2019


In [10]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match